# OS Maps Example

OS Maps Example in Python. Adapted from the tutorial [price-paid-spatial-distribution](https://github.com/OrdnanceSurvey/os-data-hub-tutorials/tree/master/data-science/price-paid-spatial-distribution) by Ordnance Survey.

### Step 0: Install requirements

If you are using this notebook directly from GitHub, all the requirements should be in `requirements.txt`. So you can just install the requirements using the provided `make` job:

```shell
make requirements
```

### Step 1: Import Python libraries

In [ ]:
import os
import fiona
import folium
import geopandas as gpd
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import requests
from datetime import datetime
from folium.plugins import FloatImage
from mpl_toolkits.axes_grid1 import make_axes_locatable
from zipfile import ZipFile

print("✅  Imported libraries")

### Step 2: Request LAD boundary from the ONS Open Geography Portal WFS

In [ ]:
# ONS Open Geography Portal WFS base path: https://ons-inspire.esriuk.com
# ONS WFS endpoint path: /arcgis/services/Administrative_Boundaries
#                        /Local_Authority_Districts_May_2020_Boundaries_UK_BFE
#                        /MapServer/WFSServer?
wfs_endpoint = ('https://ons-inspire.esriuk.com/arcgis/services/Administrative_Boundaries'
                '/Local_Authority_Districts_May_2020_Boundaries_UK_BFE/MapServer/WFSServer?')

# Define WFS parameters 
service = 'wfs'
request = 'GetFeature'
version = '2.0.0'
typeNames = ('Administrative_Boundaries_Local_Authority_Districts_May_2020_Boundaries_UK_BFE:'
             'Local_Authority_Districts__May_2020__Full_Extent_Boundaries_UK')
outputFormat = 'GEOJSON'
srsName = 'EPSG:4326'
# Define attribute-based filter using OGC WFS filter encoding specification
# Filter specifies the Government Statistical Service (GSS) Code for the Southampton local authority district area
filter = ('<ogc:Filter>'
              '<ogc:PropertyIsEqualTo>'
                  '<ogc:PropertyName>lad20cd</ogc:PropertyName>'
                  '<ogc:Literal>E06000045</ogc:Literal>'
              '</ogc:PropertyIsEqualTo>'
          '</ogc:Filter>')

# LA code for Liverpool: E08000012
# LA code for Southampton: E06000045

# Represent WFS parameters in a dictionary and collapse OGC filter into a single line
params_wfs = {'service':service, 
              'request':request,
              'version':version,
              'typeNames':typeNames,
              'outputFormat':outputFormat,
              'srsName':srsName,
              'filter':filter}

# Make HTTP GET request and raise exception if request was unsuccessful
# Turn off verification to remove the need to whitelist certificate
try:
    r = requests.get(wfs_endpoint, params=params_wfs)
    r.raise_for_status()
except requests.exceptions.RequestException as e:  
    print(e)

# Decode JSON payload returned by request    
payload = r.json()

In [ ]:
# Define coordinate reference system (CRS) codes
# WGS 84
WGS84 = 'epsg:4326'
# British National Grid
BNG = 'epsg:27700'

# Transform GeoJSON features into a GeoPandas GeoDataFrame
gdf_boundary = gpd.GeoDataFrame.from_features(payload['features'], crs=WGS84)

# Obtain polygon centroid point geometry
boundary_centroid = gdf_boundary['geometry'].centroid

# Obtain x and y coordinates of centroid point geometry
x = boundary_centroid.x
y = boundary_centroid.y

# Obtain bounds of polygon geometry
bounds = gdf_boundary['geometry'][0].bounds

# Define a OGC WFS filter compliant bounding box for the polygon geometry
# bottom-left y, bottom-left x, top-right y, top-right x
# bbox = str(bounds[1]) + ',' + str(bounds[0]) + ',' + str(bounds[3]) + ',' + str(bounds[2])
bbox = ','.join([str(bounds[1]), str(bounds[0]), str(bounds[3]), str(bounds[2])])

# Plot boundary geometry
ax = gdf_boundary.plot(color='#ff1f5b')
# Turn plot axis off
ax.axis('off')

print('✅  Transformed ONS WFS GeoJSON payload into a GeoDataFrame')

### Step 3: Construct a URL path for the OS Maps API ZXY endpoint

Note: You must generate your own OS Maps API Key at https://osdatahub.os.uk/ and store it in a .secrets file

See [#creating-a-secrets-file](https://github.com/DataS-DHSC/os-maps-example#creating-a-secrets-file) in the project README for more information

In [ ]:
# OS Data Hub base path: https://api.os.uk
# OS Maps API (ZXY) endpoint path: /maps/raster/v1/zxy/layer/{z}/{x}/{y}.png?

key = os.environ['OS_MAPS_API_KEY'] 

layer = 'Light_3857'

zxy_path = 'https://api.os.uk/maps/raster/v1/zxy/{}/{{z}}/{{x}}/{{y}}.png?key={}'.format(layer, key)

print('✅ Constructed OS Maps ZXY API path: {}'.format(zxy_path))